In [1]:
import pandas as pd
data = pd.read_excel("~/class.xlsx")
data = data.drop(['Pseudonym'], axis=1)

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
classification_pipeline = Pipeline([('scaler', StandardScaler()), ('perceptron', Perceptron())])


In [3]:
X = data.copy().drop(['Gender'], axis=1)
y = data['Gender']
classification_pipeline.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()), ('perceptron', Perceptron())])

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

#regression_pipeline = Pipeline([('regression', LinearRegression())])
#regression_pipeline = Pipeline([('scaler', StandardScaler()),  ('regression', LinearRegression())])
#regression_pipeline = Pipeline([('scaler', StandardScaler()),
#                                ('regression', LinearRegression())])


,Height,Weight,Age,Sporthours,Location
0,178.0,81,49,7,1
1,180.0,75,22,16,1
2,169.0,53,20,0,2
3,183.0,80,25,1,1
4,190.0,65,25,5,0


In [5]:
numeric_features = ["Weight", "Height", "Age", "Sporthours"]
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = ["Gender"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OrdinalEncoder(handle_unknown="ignore"))
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [14]:
data = pd.read_excel("~/class.xlsx")
data = data.drop(['Pseudonym'], axis=1)
X = data.copy().drop(["Weight"], axis=1)
y = data["Weight"].copy()

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", LinearRegression())]
)
clf.fit(X, y)


ValueError: A given column is not a column of the dataframe